# I) Summary

- The paper [Going Deeper with Convolutions][paper] introduces the first version of Inception model called GoogLeNet.


- During ILSVLC-2014, they achieved 1st place at the classification task (top-5 test error = 6.67%)


- It has around 6.7977 million parameters (without auxilaries layers) which is 9x fewer than AlexNet (ILSVRC-2012 winner) and 20x fewer than its competitor VGG-16.


- In most of the standard network architectures, the intuition is not clear why and when to perform the max-pooling operation, when to use the convolutional operation. For example, in AlextNet we have the convolutional operation and max-pooling operation following each other whereas in VGGNet, we have 3 convolutional operations in a row and then 1 max-pooling layer.


- Thus, **the idea behind GoogLeNet is to use all the operations at the same time**. It computes multiple kernels of different size over the same input map in parallel, concatenating their results into a single output. This is called an **Inception module**.

<div style="text-align: center">
    <img src="https://cdn.discordapp.com/attachments/676833120053493770/689776141221101629/unknown.png"
         height="100%" width="100%">
</div>

- Consider the following:

<div style="text-align: center">
    <img src="https://media.discordapp.net/attachments/676833120053493770/690136206092402715/unknown.png"
         height="50%" width="70%">
</div>
 

<div style="text-align: center">
    <img src="https://cdn.discordapp.com/attachments/676833120053493770/690138280058028146/unknown.png"
         height="50%" width="90%">
</div>


- The Naive approach is computationally expensive:
    - Computation cost = ((28 x 28 x 5 x 5) x 192) x 32 $\simeq$ **120 Mil**
        - We perform (28 x 28 x 5 x 5) operations along 192 channels for each of the 32 filters.


- The dimension reduction approach is **less** computationally expensive:
    - 1st layer computation cost = ((28 x 28 x 1 x 1) x 192) x 16 $\simeq$ 2.4 Mil
    - 2nd layer computation cost = ((28 x 28 x 5 x 5) x 16) x 32 $\simeq$ 10 Mil  
    - Total computation cost $\simeq$ **12.4 Mil**

---

Here its architecture:

<div style="text-align: center">
    <img src="https://cdn.discordapp.com/attachments/676833120053493770/690150147392667651/unknown.png"
         height="100%" width="100%">
</div>

- There are:
    - 9 Inception modules (red box)
    - Global Average pooling were used instead of a Fully-connected layer.
        - It enables adapting and fine-tuning on the network easily.
    - 2 auxilaries softmax layer (green box)
        - Their role is to push the network toward its goal and helps to ensure that the intermediate features are good enough for the network to learn.
        - It turns out that softmax0 and sofmax1 gives regularization effect.
        - During training, their loss gets added to the total loss with a discount weight (the losses of the auxiliary classifiers were weighted by 0.3).
        - During inference, they are discarded.
        - Structure:
            - Average pooling layer with 5×5 filter size and stride 3 resulting in an output size:
                - For 1st green box: 4x4x512.
                - For 2nd green box: 4x4x528.
            - 128 1x1 convolutions + ReLU.
            - Fully-connected layer with 1024 units + ReLU.
            - Dropout = 70%.
            - Linear layer (1000 classes) + Softmax.

<br>

<div style="text-align: center">
    <img src="https://cdn.discordapp.com/attachments/676833120053493770/690147584534511659/unknown.png"
         height="100%" width="80%">
</div>

<div style="text-align: center">
    <img src="https://cdn.discordapp.com/attachments/676833120053493770/690676447336988832/legend.png"
         height="100%" width="100%">
</div>

<div style="text-align: center">
    <img src="https://cdn.discordapp.com/attachments/676833120053493770/691604662268461056/unknown.png"
         height="100%" width="100%">
</div>

<div style="text-align: center">
    <img src="https://cdn.discordapp.com/attachments/676833120053493770/691591126502866944/unknown.png"
         height="100%" width="100%">
</div>

[paper]: https://arxiv.org/pdf/1409.4842.pdf

# II) Implementation

In [115]:
from utils import *
from classes import class_names

import numpy as np
import os
import cv2
import shutil
import urllib.request
import scipy.stats as stats
from collections import OrderedDict
from matplotlib import pyplot as plt
from IPython.display import clear_output

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets
from torchsummary import summary

## a) Loading dataset / Preprocessing

In [117]:
def load_cifar():
    
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.5], std=[0.5])])
            
    train_dataset = datasets.CIFAR10('./data', train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR10('./data', train=False, download=True, transform=transform)

    #Clear downloading message.
    clear_output()
    
    # Split dataset into training set and validation set.
    train_dataset, val_dataset = random_split(train_dataset, (45000, 5000))
    
    print("Image Shape: {}".format(train_dataset[0][0].numpy().shape), end = '\n\n')
    print("Training Set:   {} samples".format(len(train_dataset)))
    print("Validation Set:   {} samples".format(len(val_dataset)))
    print("Test Set:       {} samples".format(len(test_dataset)))
    
    # Create iterator.
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)
    
    # Delete the data/ folder.
    shutil.rmtree('./data')
    
    return train_loader, val_loader, test_loader

In [118]:
train_loader, val_loader, test_loader = load_cifar()

Image Shape: (3, 32, 32)

Training Set:   45000 samples
Validation Set:   5000 samples
Test Set:       10000 samples


## b) Architecture build

- Local Response Normalization will be replaced by Batch Normalization.

In [34]:
class ConvBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(ConvBlock, self).__init__()
        
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.ReLU()
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.act(x)
        return x

In [90]:
class InceptionModule(nn.Module):
    
    def __init__(self, in_channels, f_1x1, f_3x3_r, f_3x3, f_5x5_r, f_5x5, f_pp):
        super(InceptionModule, self).__init__()
        
        self.branch1 = nn.Sequential(
            ConvBlock(in_channels, f_1x1, kernel_size=1, stride=1, padding=0)
        )
        
        self.branch2 = nn.Sequential(
            ConvBlock(in_channels, f_3x3_r, kernel_size=1, stride=1, padding=0),
            ConvBlock(f_3x3_r, f_3x3, kernel_size=3, stride=1, padding=1)
        )
        
        self.branch3 = nn.Sequential(
            ConvBlock(in_channels, f_5x5_r, kernel_size=1, stride=1, padding=0),
            ConvBlock(f_5x5_r, f_5x5, kernel_size=5, stride=1, padding=2)
        )
        
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1, ceil_mode=True),
            ConvBlock(in_channels, f_pp, kernel_size=1, stride=1, padding=0)
        )
        
    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        
        return torch.cat([branch1, branch2, branch3, branch4], 1)

In [91]:
class InceptionAux(nn.Module):
    
    def __init__(self, in_channels, num_classes):
        super(InceptionAux, self).__init__()
        
        self.pool = nn.AdaptiveAvgPool2d((4,4))
        self.conv = nn.Conv2d(in_channels, 128, kernel_size=1, stride=1, padding=0)
        self.act = nn.ReLU()
        self.fc1 = nn.Linear(2048, 1024)
        self.dropout = nn.Dropout(0.7)
        self.fc2 = nn.Linear(1024, num_classes)
    
    def forward(self, x):
        x = self.pool(x)
        
        x = self.conv(x)
        x = self.act(x)
    
        x = torch.flatten(x, 1)
        
        x = self.fc1(x)
        x = self.act(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        
        return x

In [92]:
class GoogLeNet(nn.Module):
    
    def __init__(self, isTraining, num_classes = 1000, init_weights=True):
        super(GoogLeNet, self).__init__()
        
        self.isTraining = isTraining
        
        self.conv1 = ConvBlock(3, 64, kernel_size=7, stride=2, padding=3)
        self.pool1 = nn.MaxPool2d(3, stride=2, padding=0, ceil_mode=True)
        self.conv2 = ConvBlock(64, 64, kernel_size=1, stride=1, padding=0)
        self.conv3 = ConvBlock(64, 192, kernel_size=3, stride=1, padding=1)
        self.pool3 = nn.MaxPool2d(3, stride=2, padding=0, ceil_mode=True)
        self.inception3A = InceptionModule(in_channels=192,
                                           f_1x1=64,
                                           f_3x3_r=96,
                                           f_3x3=128,
                                           f_5x5_r=16,
                                           f_5x5=32,
                                           f_pp=32)
        self.inception3B = InceptionModule(in_channels=256,
                                           f_1x1=128,
                                           f_3x3_r=128,
                                           f_3x3=192,
                                           f_5x5_r=32,
                                           f_5x5=96,
                                           f_pp=64)
        self.pool4 = nn.MaxPool2d(3, stride=2, padding=0, ceil_mode=True)
        self.inception4A = InceptionModule(in_channels=480,
                                           f_1x1=192,
                                           f_3x3_r=96,
                                           f_3x3=208,
                                           f_5x5_r=16,
                                           f_5x5=48,
                                           f_pp=64)
        self.inception4B = InceptionModule(in_channels=512,
                                           f_1x1=160,
                                           f_3x3_r=112,
                                           f_3x3=224,
                                           f_5x5_r=24,
                                           f_5x5=64,
                                           f_pp=64)
        self.inception4C = InceptionModule(in_channels=512,
                                           f_1x1=128,
                                           f_3x3_r=128,
                                           f_3x3=256,
                                           f_5x5_r=24,
                                           f_5x5=64,
                                           f_pp=64)
        self.inception4D = InceptionModule(in_channels=512,
                                           f_1x1=112,
                                           f_3x3_r=144,
                                           f_3x3=288,
                                           f_5x5_r=32,
                                           f_5x5=64,
                                           f_pp=64)
        self.inception4E = InceptionModule(in_channels=528,
                                           f_1x1=256,
                                           f_3x3_r=160,
                                           f_3x3=320,
                                           f_5x5_r=32,
                                           f_5x5=128,
                                           f_pp=128)
        self.pool5 = nn.MaxPool2d(3, stride=2, padding=0, ceil_mode=True)
        self.inception5A = InceptionModule(in_channels=832,
                                           f_1x1=256,
                                           f_3x3_r=160,
                                           f_3x3=320,
                                           f_5x5_r=32,
                                           f_5x5=128,
                                           f_pp=128)
        self.inception5B = InceptionModule(in_channels=832,
                                           f_1x1=384,
                                           f_3x3_r=192,
                                           f_3x3=384,
                                           f_5x5_r=48,
                                           f_5x5=128,
                                           f_pp=128)
        self.pool6 = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)
        
        if self.isTraining:   
            self.aux4A = InceptionAux(512, num_classes) 
            self.aux4D = InceptionAux(528, num_classes)
        
#         if init_weights:
#             self._initialize_weights()
    
#     def _initialize_weights(self):
#         for m in self.modules():
#             print(m)
#             if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
#                 X = stats.truncnorm(-2, 2, scale=0.01)
#                 values = torch.as_tensor(X.rvs(m.weight.numel()), dtype=m.weight.dtype)
#                 values = values.view(m.weight.size())
#                 with torch.no_grad():
#                     m.weight.copy_(values)
#             elif isinstance(m, nn.BatchNorm2d):
#                 nn.init.constant_(m.weight, 1)
#                 nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.pool3(x)
        print(x.shape)
        x = self.inception3A(x)
        print(x.shape)
        x = self.inception3B(x)
        x = self.pool4(x)
        x = self.inception4A(x)
        
        if self.isTraining:
            aux1 = self.aux4A(x)
        
        x = self.inception4B(x)
        x = self.inception4C(x)
        x = self.inception4D(x)
        
        if self.isTraining:
            aux2 = self.aux4D(x)
        
        x = self.inception4E(x)
        x = self.pool5(x)
        x = self.inception5A(x)
        x = self.inception5B(x)
        x = self.pool6(x)
        x = torch.flatten(x,1)
        x = self.dropout(x)
        x = self.fc(x)
        
        if self.isTraining:
            return x, aux1, aux2
        else:
            return x

In [93]:
model = GoogLeNet(isTraining=False)

In [105]:
summary(model, (3, 224, 224))

torch.Size([2, 192, 28, 28])
torch.Size([2, 256, 28, 28])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         ConvBlock-4         [-1, 64, 112, 112]               0
         MaxPool2d-5           [-1, 64, 56, 56]               0
            Conv2d-6           [-1, 64, 56, 56]           4,160
       BatchNorm2d-7           [-1, 64, 56, 56]             128
              ReLU-8           [-1, 64, 56, 56]               0
         ConvBlock-9           [-1, 64, 56, 56]               0
           Conv2d-10          [-1, 192, 56, 56]         110,784
      BatchNorm2d-11          [-1, 192, 56, 56]             384
             ReLU-12          [-1, 192, 56, 56]               0
        ConvBlock-13          [-1, 192, 56, 5